In [8]:
import pandas as pd
import csv
import sqlite3
from sqlite3 import Error
import os
from sklearn.model_selection import train_test_split

In [9]:
def create_connection(db_file, delete_db=False):
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [10]:
train_titanic = pd.read_csv(r'C:\Users\bsham\Downloads\train.csv')
db_file = 'titanic.db'
conn = create_connection(db_file, delete_db=True)
cur = conn.cursor()
sql_create = """CREATE TABLE IF NOT EXISTS train (PassengerID PRIMARY KEY NOT NULL, 
                                                Survived, Pclass, Name, Sex, Age, SibSp, Parch, 
                                                Ticket, Fare, Cabin, Embarked);"""
create_table(conn,sql_create)
count = 0

with open(r'C:\Users\bsham\Downloads\train.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        count = count + 1
        if count>1:
            #print(row)
            sql_val = """INSERT INTO train(PassengerID, 
                                                Survived, Pclass, Name, Sex, Age, SibSp, Parch, 
                                                Ticket, Fare, Cabin, Embarked) VALUES(?,?,?,?,?,?,?,?,?,?,?,?)"""
            val = (row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11])
            cur.execute(sql_val,val)
            conn.commit()

In [11]:
cur = conn.cursor()
try:
    cur.execute('SELECT * FROM "train"')
except:
    print ("I can't SELECT from train")
    
titanic = pd.DataFrame(cur.fetchall(),columns=['PassengerID', 
                                                'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 
                                                'Ticket', 'Fare', 'Cabin', 'Embarked'])

cur.close()

conn.close()

In [12]:
titanic_survived = train_titanic['Survived']


titanic = titanic.drop("Survived", axis=1)

In [15]:
import pandas as pd
import csv
import sqlite3
from sqlite3 import Error
import os
from sklearn.model_selection import train_test_split
from sklearn import svm, preprocessing
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(titanic, titanic_survived, test_size=0.3, random_state=0)


train_test_data = [X_train, X_test]

#get the title of the passenger such as Mr, Mrs etc from the name

for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

#map the passenger titles to a value
title_mapping = {"Mr": 0, "Miss": 1, "Mrs": 2, 
                 "Master": 3, "Dr": 4, "Rev": 4, "Col": 4, "Major": 4, "Mlle": 4,"Countess": 4,
                 "Ms": 4, "Lady": 4, "Jonkheer": 4, "Don": 4, "Dona" : 4, "Mme": 4,"Capt": 4,"Sir": 4 }
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    

X_train.drop('Name', axis = 1, inplace = True)
X_test.drop('Name', axis = 1, inplace = True)

#map male and female values to 0 and 1
sex_mapping = {"male": 0, "female": 1}
for dataset in train_test_data:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)

    #incomplete values in age column are filled with median age values for each title group -
    #ex: median age for people with Mr. in their name - 

X_train['Age'] = pd.to_numeric(X_train['Age'], errors='coerce')
X_test['Age'] = pd.to_numeric(X_test['Age'], errors='coerce')

X_train["Age"].fillna(X_train.groupby("Title")["Age"].transform("median"), inplace=True)
X_test["Age"].fillna(X_test.groupby("Title")["Age"].transform("median"), inplace=True)

#map values for embarked and fill missing values with median values
embarked_mapping = {"S":0, "C":1, "Q": 2}
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].map(embarked_mapping)
    


X_train["Embarked"].fillna(X_train["Embarked"].median(), inplace=True)
X_test["Embarked"].fillna(X_test["Embarked"].median(), inplace=True)

X_train["Embarked"] = X_train["Embarked"].astype(int)
X_test["Embarked"] = X_test["Embarked"].astype(int)

X_train['Fare'] = pd.to_numeric(X_train['Fare'], errors='coerce')
X_test['Fare'] = pd.to_numeric(X_test['Fare'], errors='coerce')

X_train["Fare"].fillna(X_train.groupby("Pclass")["Fare"].transform("median"), inplace=True)
X_test["Fare"].fillna(X_test.groupby("Pclass")["Fare"].transform("median"), inplace=True)

for dataset in train_test_data:
    dataset['Cabin'] = dataset['Cabin'].str[:1]

cabin_mapping = {"A": 0, "B": 0.4, "C": 0.8, "D": 1.2, "E": 1.6, "F": 2, "G": 2.4, "T": 2.8}
for dataset in train_test_data:
    dataset['Cabin'] = dataset['Cabin'].map(cabin_mapping)

X_train["Cabin"].fillna(X_train.groupby("Pclass")["Cabin"].transform("median"), inplace=True)
X_test["Cabin"].fillna(X_test.groupby("Pclass")["Cabin"].transform("median"), inplace=True)

for dataset in train_test_data:
    dataset.loc[ dataset['Fare'] <= 17, 'Fare'] = 0,
    dataset.loc[(dataset['Fare'] > 17) & (dataset['Fare'] <= 30), 'Fare'] = 1,
    dataset.loc[(dataset['Fare'] > 30) & (dataset['Fare'] <= 100), 'Fare'] = 2,
    dataset.loc[ dataset['Fare'] > 100, 'Fare'] = 3

X_train['SibSp'] = pd.to_numeric(X_train['SibSp'], errors='coerce')
X_test['SibSp'] = pd.to_numeric(X_test['SibSp'], errors='coerce')
X_train['Parch'] = pd.to_numeric(X_train['Parch'], errors='coerce')
X_test['Parch'] = pd.to_numeric(X_test['Parch'], errors='coerce')



X_train["FamilySize"] = X_train["SibSp"]  +  X_train["Parch"] + 1
X_test["FamilySize"] = X_test["SibSp"] + X_test["Parch"] + 1

family_mapping = {1: 0, 2: 0.4, 3: 0.8, 4: 1.2, 5: 1.6, 6: 2, 7: 2.4, 8: 2.8, 9: 3.2, 10: 3.6, 11: 4}
for dataset in train_test_data:
    dataset['FamilySize'] = dataset['FamilySize'].map(family_mapping)

features_drop = ['Ticket', 'SibSp', 'Parch']
X_train = X_train.drop(features_drop, axis=1)
X_test = X_test.drop(features_drop, axis=1)

#X_train = X_train.drop('Survived', axis=1)
#X_test = X_test.drop('Survived',axis = 1)
X_train.head()
X_test.head()











C:\Users\bsham\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\bsham\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\bsham\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

,PassengerID,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,FamilySize
495,496,3,0,28.0,0.0,2.0,1,0,0.0
648,649,3,0,28.0,0.0,2.0,0,0,0.0
278,279,3,0,7.0,1.0,2.0,2,3,2.0
31,32,1,1,34.5,3.0,0.4,1,2,0.4
255,256,3,1,29.0,0.0,2.0,1,2,0.8


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

# Logistic Regression
log_reg = LogisticRegression(random_state=0)
log_reg.fit(X_train, y_train)

# Predicting using validation set
log_reg_pred = log_reg.predict(X_test)

print(log_reg_pred)
log_reg_acc = accuracy_score(y_test, log_reg_pred)

print('''Accuracy score: \n{}'''.format( log_reg_acc))

[0 0 0 1 1 0 1 1 0 1 0 1 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 1 1 0 0 1 0 0 0 1 0
 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 0 0 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 0 1 1 0 0 1 1 0 0 1 0 1 1 0 1 0 0 1 1 1 0 0 1 0
 1 0 1 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 1 1 1 0 1
 1 0 0 1 1 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0
 0 1 0 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 0 1
 1 0 0 0 0 1 0 0 0 1 1 1 0 0 1 1 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0
 0 0 0 0 0 0 0 1 0]
Accuracy score: 
0.7910447761194029


C:\Users\bsham\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
